In [ ]:
import json
import numpy as np
import string
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q datasets sentencepiece

!pip uninstall -y -q transformers accelerate
!pip install -q transformers accelerate

!pip install -q evaluate

!pip install -q git+https://github.com/huggingface/accelerate

!git clone https://github.com/huggingface/transformers.git
!cd transformers
!pip install -e.

!pip install -q parsivar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
from datasets import load_dataset

# data_files = {"train": "/content/drive/MyDrive/train.json", "validation": "/content/drive/MyDrive/dev.json"}

train_path = "/content/drive/MyDrive/extra_translations/ansarian/ansarian_train.json"
valdiation_path = "/content/drive/MyDrive/extra_translations/ansarian/ansarian_dev.json"

data_files = {"train": train_path, "validation": valdiation_path}
dataset = load_dataset("json", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
max_length = 384
stride = 128

def preprocess_validation_examples(examples, tokenizer):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["pq_id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
!pip install -q parsivar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 23.4 MB/s eta 0:00:00


In [ ]:
import json
import torch
from parsivar import Normalizer

my_normalizer = Normalizer()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open('/content/drive/MyDrive/extra_translations/ansarian/ansarian_dev.json', 'r') as f:
  dev_data_list = json.load(f)

predicted_output = []
ground_truth = []

for qa in dev_data_list[:100]:
  # ground_truth.append(qa['answers'][0]['text'])
  ground_truth.append(my_normalizer.normalize(qa['answers'][0]['text']))

  # Use the model to answer the question
  result = model(question=qa['question'], context=qa['context'])

  answer = result["answer"]

  predicted_output.append(my_normalizer.normalize(answer))


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

checkpoint1 = '/content/drive/MyDrive/parsbert_pquad/checkpoint-8000'
checkpoint2 = '/content/drive/MyDrive/albert_pquad/checkpoint-8000'

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

tokenizer1 = AutoTokenizer.from_pretrained(checkpoint1)
model1 = AutoModelForQuestionAnswering.from_pretrained(checkpoint1).to(device)

tokenizer2 = AutoTokenizer.from_pretrained(checkpoint2)
model2 = AutoModelForQuestionAnswering.from_pretrained(checkpoint2).to(device)


In [ ]:
from parsivar import Normalizer

my_normalizer = Normalizer()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

def compute_answer(checkpoint_list, data_path = '/content/drive/MyDrive/extra_translations/ansarian/ansarian_dev.json'):

  with open(data_path, 'r') as f:
    dev_data_list = json.load(f)

  models_results = []
  theoretical_answers = []

  for checkpoint in checkpoint_list:
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForQuestionAnswering.from_pretrained(checkpoint).to(device)

    predicted_outputs = []

    for qa in dev_data_list:
      question = qa['question']
      context = qa['context']

      inputs = tokenizer(question, context, return_tensors='pt')

      with torch.no_grad():
        outputs = model(**inputs)

      start_logits = outputs.start_logits
      end_logits = outputs.end_logits

      start_probs = torch.nn.functional.softmax(start_logits, dim=1)
      end_probs = torch.nn.functional.softmax(end_logits, dim=1)

      # Find spans with highest probabilities
      n_best = 20
      max_answer_length = 30
      try:
        start_prob, start_index = torch.topk(start_probs, n_best, dim=1)  # Top 5 start positions
        end_prob, end_index = torch.topk(end_probs, n_best, dim=1)  # Top 5 end positions
      except:
        n_best = 15

      answers_dict = {}
      # Extract text spans
      for i in range(n_best):
          start_pos = start_index[0][i]
          end_pos = end_index[0][i]
          if end_pos < start_pos or end_pos - start_pos > max_answer_length:
              continue
          answer_tokens = inputs['input_ids'][0][start_pos: end_pos+1]
          answer = tokenizer.decode(answer_tokens)
          if answer and answer != '[CLS]' and answer != '[SEP]' and answer in context and len(answer) <= 30 and len(answer)>0:
            answers_dict[answer] = start_prob[0][i].item()

      try:
        final_answer = max(answers_dict, key=answers_dict.get)
        predicted_outputs.append({"id": qa['pq_id'], "prediction_text": my_normalizer.normalize(final_answer).strip(), "score": answers_dict[final_answer]})
      except:
        final_answer = ''
        predicted_outputs.append({"id": qa['pq_id'], "prediction_text": my_normalizer.normalize(final_answer).strip(), "score": 0})

    models_results.append(predicted_outputs)

  predicted_answers = []

  for i in range(len(dev_data_list)):
    max = 0
    answer = ''
    for result in models_results:
      if result[i]['score'] > max:
        max = result[i]['score']
        answer = result[i]['prediction_text']

    predicted_answers.append({"id": dev_data_list[i]['pq_id'], "prediction_text": answer})


  # print(predicted_answers)


  for qa in dev_data_list:
    new_ex = {}
    new_ex['id'] = qa["pq_id"]
    new_ex['answers'] = {}
    new_ex['answers']['answer_start'] = [qa['answers'][0]['start_char']]
    # new_ex['answers']['text'] = [qa['answers'][0]['text']]
    new_ex['answers']['text'] = [my_normalizer.normalize(qa['answers'][0]['text'])]
    theoretical_answers.append(new_ex)


  return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import evaluate

metric = evaluate.load("squad")

checkpoints = ['/content/drive/MyDrive/albert_pquad/checkpoint-8000',
               '/content/drive/MyDrive/parsbert_pquad/checkpoint-8000']

compute_answer(checkpoints)

[{'id': 's2.283', 'prediction_text': ''}, {'id': 's2.34', 'prediction_text': ''}, {'id': 's2.36', 'prediction_text': ''}, {'id': 's7.22', 'prediction_text': ''}, {'id': 's7.20', 'prediction_text': ''}, {'id': 's6.161', 'prediction_text': ''}, {'id': 's41.15', 'prediction_text': ''}, {'id': 's46.21', 'prediction_text': ''}, {'id': 's46.21', 'prediction_text': ''}, {'id': 's51.38', 'prediction_text': ''}, {'id': 's51.38', 'prediction_text': ''}, {'id': 's54.22', 'prediction_text': ''}, {'id': 's54.18', 'prediction_text': ''}, {'id': 's69.6', 'prediction_text': ''}, {'id': 's69.5', 'prediction_text': ''}, {'id': 's73.18', 'prediction_text': ''}, {'id': 's73.17', 'prediction_text': ''}, {'id': 's73.16', 'prediction_text': ''}, {'id': 's33.33', 'prediction_text': ''}, {'id': 's33.53', 'prediction_text': ''}, {'id': 's33.56', 'prediction_text': ''}, {'id': 's2.153', 'prediction_text': ''}, {'id': 's2.151', 'prediction_text': ''}, {'id': 's3.32', 'prediction_text': ''}, {'id': 's3.164', 'pred

{'exact_match': 0.0, 'f1': 0.0}

In [ ]:
from parsivar import Normalizer

my_normalizer = Normalizer()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


# def compute_answer(checkpoint_list, data_path = '/content/drive/MyDrive/extra_translations/ansarian/ansarian_dev.json'):

data_path = '/content/drive/MyDrive/extra_translations/ansarian/ansarian_dev.json'

checkpoint_list = ['/content/drive/MyDrive/albert_pquad/checkpoint-8000',
               '/content/drive/MyDrive/parsbert_pquad/checkpoint-8000']


with open(data_path, 'r') as f:
  dev_data_list = json.load(f)

models_results = []
theoretical_answers = []

for checkpoint in checkpoint_list:
  tokenizer = AutoTokenizer.from_pretrained(checkpoint)
  model = AutoModelForQuestionAnswering.from_pretrained(checkpoint).to(device)

  predicted_outputs = []

  for qa in dev_data_list:
    question = qa['question']
    context = qa['context']

    inputs = tokenizer(question, context, return_tensors='pt')

    with torch.no_grad():
      outputs = model(**inputs)

    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    start_probs = torch.nn.functional.softmax(start_logits, dim=1)
    end_probs = torch.nn.functional.softmax(end_logits, dim=1)

    # Find spans with highest probabilities
    n_best = 20
    max_answer_length = 30
    try:
      start_prob, start_index = torch.topk(start_probs, n_best, dim=1)  # Top 5 start positions
      end_prob, end_index = torch.topk(end_probs, n_best, dim=1)  # Top 5 end positions
    except:
      n_best = 15

    answers_dict = {}
    # Extract text spans
    for i in range(n_best):
        start_pos = start_index[0][i]
        end_pos = end_index[0][i]
        if end_pos < start_pos or end_pos - start_pos > max_answer_length:
            continue
        answer_tokens = inputs['input_ids'][0][start_pos: end_pos+1]
        answer = tokenizer.decode(answer_tokens)
        if answer and answer != '[CLS]' and answer != '[SEP]' and answer in context and len(answer) <= 30 and len(answer)>0:
          answers_dict[answer] = start_prob[0][i].item()

    try:
      final_answer = max(answers_dict, key=answers_dict.get)
      predicted_outputs.append({"id": qa['pq_id'], "prediction_text": my_normalizer.normalize(final_answer).strip(), "score": answers_dict[final_answer]})
    except:
      final_answer = ''
      predicted_outputs.append({"id": qa['pq_id'], "prediction_text": my_normalizer.normalize(final_answer).strip(), "score": 0})

  models_results.append(predicted_outputs)

predicted_answers = []

for i in range(len(dev_data_list)):
  max = 0
  answer = ''
  for result in models_results:
    if result[i]['score'] > max:
      max = result[i]['score']
      answer = result[i]['prediction_text']

  predicted_answers.append({"id": dev_data_list[i]['pq_id'], "prediction_text": answer})


# print(predicted_answers)


for qa in dev_data_list:
  new_ex = {}
  new_ex['id'] = qa["pq_id"]
  new_ex['answers'] = {}
  new_ex['answers']['answer_start'] = [qa['answers'][0]['start_char']]
  # new_ex['answers']['text'] = [qa['answers'][0]['text']]
  new_ex['answers']['text'] = [my_normalizer.normalize(qa['answers'][0]['text'])]
  theoretical_answers.append(new_ex)


metric.compute(predictions=predicted_answers, references=theoretical_answers)

{'exact_match': 49.390243902439025, 'f1': 60.497483546264014}

In [ ]:
import json
import torch
from parsivar import Normalizer

my_normalizer = Normalizer()

with open('/content/drive/MyDrive/extra_translations/ansarian/ansarian_dev.json', 'r') as f:
  dev_data_list = json.load(f)

predicted_answers = []
theoretical_answers = []

for qa in dev_data_list:

  ground_truth.append(my_normalizer.normalize(qa['answers'][0]['text']))
  # ground_truth.append(qa['answers'][0]['text'])

  question = qa['question']
  context = qa['context']

  inputs1 = tokenizer1(question, context, return_tensors='pt')
  inputs2 = tokenizer2(question, context, return_tensors='pt')

  # Get model output
  with torch.no_grad():
    outputs1 = model1(**inputs1)
    outputs2 = model2(**inputs2)

  start_logits1 = outputs1.start_logits
  end_logits1 = outputs1.end_logits

  start_logits2 = outputs2.start_logits
  end_logits2 = outputs2.end_logits

  # Apply softmax
  start_probs1 = torch.nn.functional.softmax(start_logits1, dim=1)
  end_probs1 = torch.nn.functional.softmax(end_logits1, dim=1)

  start_probs2 = torch.nn.functional.softmax(start_logits2, dim=1)
  end_probs2 = torch.nn.functional.softmax(end_logits2, dim=1)

  # Find spans with highest probabilities
  n_best = 20
  max_answer_length = 30
  try:
    start_prob1, start_index1 = torch.topk(start_probs, n_best, dim=1)  # Top 5 start positions
    end_prob1, end_index1 = torch.topk(end_probs, n_best, dim=1)  # Top 5 end positions

    start_prob, start_index = torch.topk(start_probs, n_best, dim=1)  # Top 5 start positions
    end_prob, end_index = torch.topk(end_probs, n_best, dim=1)  # Top 5 end positions
  except:
    n_best = 15

  answers_dict = {}
  # Extract text spans
  for i in range(n_best):
      start_pos = start_index[0][i]
      end_pos = end_index[0][i]
      if end_pos < start_pos or end_pos - start_pos > max_answer_length:
          continue
      answer_tokens = inputs['input_ids'][0][start_pos: end_pos+1]
      answer = tokenizer.decode(answer_tokens)
      if answer and answer != '[CLS]' and answer != '[SEP]' and answer in context and len(answer) <= 30 and len(answer)>0:
        answers_dict[answer] = start_prob[0][i].item()

  try:
    final_answer = max(answers_dict, key=answers_dict.get)
  except:
    final_answer = ''
  predicted_answers.append({"id": qa['pq_id'], "prediction_text": final_answer})
  # predicted_answers.append({"id": qa['pq_id'], "prediction_text": my_normalizer.normalize(final_answer).strip()})
  predicted_output.append(my_normalizer.normalize(final_answer).strip())



  new_ex = {}
  new_ex['id'] = qa["pq_id"]
  new_ex['answers'] = {}
  new_ex['answers']['answer_start'] = [qa['answers'][0]['start_char']]
  new_ex['answers']['text'] = [qa['answers'][0]['text']]
  # new_ex['answers']['text'] = [my_normalizer.normalize(qa['answers'][0]['text'])]
  theoretical_answers.append(new_ex)
  # print(final_answer)
  # print("***********************")



In [ ]:
import json
import torch
from parsivar import Normalizer

my_normalizer = Normalizer()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open('/content/drive/MyDrive/extra_translations/ansarian/ansarian_dev.json', 'r') as f:
  dev_data_list = json.load(f)

predicted_output = []
ground_truth = []

for qa in dev_data_list[:100]:
  # ground_truth.append(qa['answers'][0]['text'])
  ground_truth.append(my_normalizer.normalize(qa['answers'][0]['text']))

  # Use the model to answer the question
  result = model(question=qa['question'], context=qa['context'])

  answer = result["answer"]

  predicted_output.append(my_normalizer.normalize(answer))


In [ ]:
import json
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
from parsivar import Normalizer


tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/parsbert_pquad/checkpoint-8000")
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = AutoModelForQuestionAnswering.from_pretrained("/content/drive/MyDrive/parsbert_pquad/checkpoint-8000").to(device)
my_normalizer = Normalizer()


with open('/content/drive/MyDrive/extra_translations/ansarian/ansarian_dev.json', 'r') as f:
  dev_data_list = json.load(f)

predicted_output = []
ground_truth = []

predicted_answers = []
theoretical_answers = []

for qa in dev_data_list:

  ground_truth.append(my_normalizer.normalize(qa['answers'][0]['text']))
  # ground_truth.append(qa['answers'][0]['text'])

  question = qa['question']
  context = qa['context']

  inputs = tokenizer(question, context, return_tensors='pt')

  # Get model output
  with torch.no_grad():
    outputs = model(**inputs)

  start_logits = outputs.start_logits
  end_logits = outputs.end_logits

  # Apply softmax
  start_probs = torch.nn.functional.softmax(start_logits, dim=1)
  end_probs = torch.nn.functional.softmax(end_logits, dim=1)
  # print(start_probs)
  # print(len(start_probs[0]))
  # break

  # Find spans with highest probabilities
  n_best = 20
  max_answer_length = 30
  try:
    start_prob, start_index = torch.topk(start_probs, n_best, dim=1)  # Top 5 start positions
    end_prob, end_index = torch.topk(end_probs, n_best, dim=1)  # Top 5 end positions
  except:
    n_best = 15
  # print(start_prob)
  # print(start_index)
  # print(start_prob[0][1].item())
  # break

  answers_dict = {}
  # Extract text spans
  for i in range(n_best):
      start_pos = start_index[0][i]
      end_pos = end_index[0][i]
      if end_pos < start_pos or end_pos - start_pos > max_answer_length:
          continue
      answer_tokens = inputs['input_ids'][0][start_pos: end_pos+1]
      # answer_tokens = tokenizer.convert_ids_to_tokens(answer_tokens, skip_special_tokens=True)
      # answer = tokenizer.decode(tokenizer.convert_tokens_to_string(answer_tokens))
      answer = tokenizer.decode(answer_tokens)
      if answer and answer != '[CLS]' and answer != '[SEP]' and answer in context and len(answer) <= 30 and len(answer)>0:
        answers_dict[answer] = start_prob[0][i].item()

      # print(f"Answer {i+1}: {answer}")
  try:
    final_answer = max(answers_dict, key=answers_dict.get)
  except:
    final_answer = ''
  # predicted_answers.append({"id": qa['pq_id'], "prediction_text": final_answer})
  predicted_answers.append({"id": qa['pq_id'], "prediction_text": my_normalizer.normalize(final_answer).strip()})
  predicted_output.append(my_normalizer.normalize(final_answer).strip())



  new_ex = {}
  new_ex['id'] = qa["pq_id"]
  new_ex['answers'] = {}
  new_ex['answers']['answer_start'] = [qa['answers'][0]['start_char']]
  # new_ex['answers']['text'] = [qa['answers'][0]['text']]
  new_ex['answers']['text'] = [my_normalizer.normalize(qa['answers'][0]['text'])]
  theoretical_answers.append(new_ex)
  # print(final_answer)
  # print("***********************")



In [ ]:
predicted_answers

[{'id': 's2.283', 'prediction_text': 'شهادت را پنهان نکنید'},
 {'id': 's2.34', 'prediction_text': 'ابلیس'},
 {'id': 's2.36', 'prediction_text': 'شیطان'},
 {'id': 's7.22', 'prediction_text': 'شیطان'},
 {'id': 's7.20', 'prediction_text': 'شیطان'},
 {'id': 's6.161', 'prediction_text': 'ابراهیم'},
 {'id': 's41.15', 'prediction_text': 'قوم عاد'},
 {'id': 's46.21', 'prediction_text': 'قوم عاد'},
 {'id': 's46.21', 'prediction_text': 'قوم عاد'},
 {'id': 's51.38', 'prediction_text': 'موسی'},
 {'id': 's51.38', 'prediction_text': 'فرعون'},
 {'id': 's54.22', 'prediction_text': 'برای پند گرفتن'},
 {'id': 's54.18', 'prediction_text': 'عاد'},
 {'id': 's69.6', 'prediction_text': 'قوم عاد'},
 {'id': 's69.5', 'prediction_text': 'قوم ثمود'},
 {'id': 's73.18', 'prediction_text': 'درهم'},
 {'id': 's73.17', 'prediction_text': 'کودکان'},
 {'id': 's73.16', 'prediction_text': 'فرعون'},
 {'id': 's33.33', 'prediction_text': 'شما اهل بیت'},
 {'id': 's33.53', 'prediction_text': 'هنگامی که دعوت شدید وارد شوید'},
 {

In [ ]:
import evaluate

metric = evaluate.load("squad")

a = metric.compute(predictions=predicted_answers, references=theoretical_answers)

a

{'exact_match': 42.68292682926829, 'f1': 53.59514130855593}